# Target Description Language

In [1]:
from target import Target, TargetList

In [2]:
# Example: fully specify target.  Coordinates in decimal degrees.
t1 = Target(name='M42', RA=82.5, Dec=-5, frame='icrs',
            PMRA=10, PMDec=-10, epoch=1975.5,
            rotmode='PA', PA=90,
            objecttype='science',
            RAOffset=8.4, DecOffset=-4.5,
            acquisition='guider: offset',
            mag={'V': 4.3, 'K': 6.7},
            comment='Star formation is cool',
            wrap='shortest',
           )
t1

M42              05 30 29.61 -05 07 22.34 2000 rotmode=PA PA=90.0 raoff=8.4 decoff=-4.5 wrap=shortest vmag=4.30 # Vmag=4.30 Kmag=6.70 Star formation is cool

In [3]:
# Example: fully specify target.  Coordinates in hmsdms.
t2 = Target(name='M31', RA='00:42:44.33', Dec='41:16:07.50',
            rotmode='PA', PA=32,
            RAOffset=12.1, DecOffset=-1.9,
            objecttype='science',
            acquisition='guider: offset',
            mag={'V': 2.3, 'K': 8.7},
            comment='Galaxies are boring',
            wrap='north',
           )
t2

M31              00 42 44.33 +41 16 07.50 2000 rotmode=PA PA=32.0 raoff=12.1 decoff=-1.9 wrap=north vmag=2.30 # Vmag=2.30 Kmag=8.70 Galaxies are boring

In [4]:
# Example: Resolve target by name.
t3 = Target('NGC1333',
            rotmode='PA', PA=22.5,
            objecttype='science',
            acquisition='blind',
            comment='Low mass star formation is the coolest')
t3

NGC1333          03 29 08.95 +31 22 07.68 2000 rotmode=PA PA=22.5 # Low mass star formation is the coolest

In [5]:
# Example: Resolve target by name, set vertical angle mode, set obstype
t4 = Target('Feige110',
            objecttype='flux standard',
            rotmode='vertical',
            acquisition='guider: bright',
            comment='I guess we need a standard')
t4

/Users/joshw/git/KeckODL/target.py:259: TargetWarning: No PA given, assuming 0 deg
  warn('No PA given, assuming 0 deg', category=TargetWarning)


Feige110         23 19 58.40 -05 09 56.17 2000 rotmode=vertical PA=0.0 # I guess we need a standard

In [6]:
# Build a target list, write as TDL TAML file and as starlist
tl = TargetList([t1, t2, t3, t4])
tl.write('targets.txt')
tl.write_starlist('starlist.txt')
tl

M42              05 30 29.61 -05 07 22.34 2000 rotmode=PA PA=90.0 raoff=8.4 decoff=-4.5 wrap=shortest vmag=4.30 # Vmag=4.30 Kmag=6.70 Star formation is cool
M31              00 42 44.33 +41 16 07.50 2000 rotmode=PA PA=32.0 raoff=12.1 decoff=-1.9 wrap=north vmag=2.30 # Vmag=2.30 Kmag=8.70 Galaxies are boring
NGC1333          03 29 08.95 +31 22 07.68 2000 rotmode=PA PA=22.5 # Low mass star formation is the coolest
Feige110         23 19 58.40 -05 09 56.17 2000 rotmode=vertical PA=0.0 # I guess we need a standard

In [7]:
# Read in a target list from the TDL YAML file
newtl = TargetList().read('targets.txt')
newtl

M42              05 30 29.61 -05 07 22.32 2000 rotmode=PA PA=90.0 raoff=8.4 decoff=-4.5 vmag=4.30 # Kmag=6.70 Vmag=4.30 Star formation is cool
M31              00 42 44.33 +41 16 07.50 2000 rotmode=PA PA=32.0 raoff=12.1 decoff=-1.9 vmag=2.30 # Kmag=8.70 Vmag=2.30 Galaxies are boring
NGC1333          03 29 08.95 +31 22 07.68 2000 rotmode=PA PA=22.5 # Low mass star formation is the coolest
Feige110         23 19 58.40 -05 09 56.17 2000 rotmode=vertical PA=0.0 # I guess we need a standard

In [8]:
from astropy.time import Time
tl.set_obstime(Time(2045.11, format='decimalyear'))
tl

M42              05 30 45.44 -05 11 18.69 2000 rotmode=PA PA=90.0 raoff=8.4 decoff=-4.5 wrap=shortest vmag=4.30 # Vmag=4.30 Kmag=6.70 Star formation is cool
M31              00 42 44.33 +41 16 07.50 2000 rotmode=PA PA=32.0 raoff=12.1 decoff=-1.9 wrap=north vmag=2.30 # Vmag=2.30 Kmag=8.70 Galaxies are boring
NGC1333          03 29 08.95 +31 22 07.68 2000 rotmode=PA PA=22.5 # Low mass star formation is the coolest
Feige110         23 19 58.40 -05 09 56.17 2000 rotmode=vertical PA=0.0 # I guess we need a standard

# Observing Sequences

## KCWI Example

In [9]:
from detector_config import KCWIblueDetectorConfig
from instrument_config import KCWIblueConfig
from offset import ABBA, Stare, StarSkyStar
from sequence import SequenceElement, Sequence

In [10]:
kcwi_1800s = KCWIblueDetectorConfig(exptime=1800, binning='2x2')
print(kcwi_1800s)

1800.0s (10)


In [11]:
med_slicer_4800 = KCWIblueConfig(slicer='medium', grating='BH3', cwave=4800)
print(med_slicer_4800)

medium BH3 4800 A


In [12]:
skyoffset = StarSkyStar(dx=30, dy=30)
print(skyoffset)

StarSkyStar (30 30)


In [13]:
myseq = Sequence([SequenceElement(pattern=skyoffset,
                                  detconfig=kcwi_1800s,
                                  instconfig=med_slicer_4800,
                                  repeat=5),
                  ])
print(myseq)

Pattern            |DetectorConfig               |InstrumentConfig             |repeat
-------------------|-----------------------------|-----------------------------|------
StarSkyStar (30 30)|1800.0s (10)                 |medium BH3 4800 A            |     5


## MOSFIRE Example

In [14]:
from detector_config import MOSFIREDetectorConfig
from instrument_config import MOSFIREConfig
from offset import ABBA
from sequence import SequenceElement, Sequence

In [15]:
# detector configs
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
print(mosfire_180s)
print(mosfire_120s)

180.0s (MCDS16, 1 coadds)
120.0s (MCDS16, 1 coadds)


In [16]:
# instrument configs
mosfire_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='my_MOS_mask')
mosfire_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='my_MOS_mask')
mosfire_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='my_MOS_mask')
mosfire_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='my_MOS_mask')
print(mosfire_Y)
print(mosfire_J)
print(mosfire_H)
print(mosfire_K)

my_MOS_mask Y-spectroscopy
my_MOS_mask J-spectroscopy
my_MOS_mask H-spectroscopy
my_MOS_mask K-spectroscopy


In [17]:
abba = ABBA(offset=1.5)
print(abba)

ABBA (1.5)


In [18]:
abba

 dx(")| dy(")| dr(deg)
------|------|--------
  +0.0|  +1.5|    +0.0
  +0.0|  -1.5|    +0.0
  +0.0|  -1.5|    +0.0
  +0.0|  +1.5|    +0.0

In [19]:
# sequence
myseq_mosfire = Sequence([SequenceElement(pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_Y, repeat=5),
                          SequenceElement(pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_J, repeat=8),
                          SequenceElement(pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_H, repeat=8),
                          SequenceElement(pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_K, repeat=5),
                         ])
print(myseq_mosfire)

Pattern            |DetectorConfig               |InstrumentConfig             |repeat
-------------------|-----------------------------|-----------------------------|------
ABBA (1.5)         |180.0s (MCDS16, 1 coadds)    |my_MOS_mask Y-spectroscopy   |     5
ABBA (1.5)         |120.0s (MCDS16, 1 coadds)    |my_MOS_mask J-spectroscopy   |     8
ABBA (1.5)         |120.0s (MCDS16, 1 coadds)    |my_MOS_mask H-spectroscopy   |     8
ABBA (1.5)         |180.0s (MCDS16, 1 coadds)    |my_MOS_mask K-spectroscopy   |     5
